# Tutorial 5 -- Human Detection 
This tutorial will show you how to detect human with the robot. We will use multi-thread to capture the color images and depth images. Then the ssd mobilenet v2 nerual network structure is used for human detection. To enable real-time performance, the tensorRT deep learning framework is used. More information can be found in the nvidia's jetbot project website: https://github.com/NVIDIA-AI-IOT/jetbot

### Step 1 load the pre-trained object detection model

Before running the program, we need to download the pre-trained detection model and put it in the current folder. The model can be found in the following link:https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view

In [1]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)
    
# TODO: INVESTIGATE OTHER HUMAN DETECTORS
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')


### Step 2 Initialize the camera instance for the Intel realsense sensor D435i

In [2]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of these two values will be captured by traitlets
    color_value = traitlets.Any()
    depth_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
    
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        self.stop_flag = False
        
        #start capture the first depth and color image
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        self.depth_image = depth_image
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap    

    def _capture_frames(self):
        while(self.stop_flag==False):
            frames = self.pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            image = np.asanyarray(color_frame.get_data())
            self.color_value = image

            depth_frame = frames.get_depth_frame()              
            depth_image = np.asanyarray(depth_frame.get_data())
            self.depth_image = depth_image
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap
    
    def start(self):
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        self.stop_flag = True
        if hasattr(self, 'thread'):
            self.thread.join()        
        if self.pipeline_started:
            self.pipeline.stop()

def bgr8_to_jpeg(value,quality=75):
    return bytes(cv2.imencode('.jpg',value)[1])

In [3]:
# create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

### Step 3 Perform object detection and display the results on widgets
Human is labeled is 1 in the pretrained model. A full list of the detection class indices can be found in the following link https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt

The following program will ask the robot to move forward if a human is detected in the captured color image.

In [11]:
import time
from RobotClass import Robot

robot = Robot()


camera.unobserve_all()
time.sleep(1.0)
robot.stop()

import ipywidgets.widgets as widgets
from IPython.display import display, HTML

width = 640
height = 480

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
display(widgets.VBox([
    widgets.HBox([image_widget,]),
    label_widget
]))

DEPTH_THRESHOLD = 2000
DEPTH_VALUE_THRESHOLD = 10
VARIANCE_THRESH = 0.1
DEPTH_THRESHOLD_MIN = DEPTH_THRESHOLD * (1-VARIANCE_THRESH)
DEPTH_THRESHOLD_MAX = 2000 * (1+VARIANCE_THRESH)
# TODO: EXPERIMENT WITH THRESHOLD TO SEE HOW IT AFFECTS TURNING
SIDE_DIFF_TURN_THRESHOLD = 0.1
FOLLOW_SPEED = 0.3
FOLLOW_TURN_SPEED = 0.5

#initialize the Robot class
def processing(change):
#     SHOWN IMAGE
    IMAGE = camera.depth_value
    
    image = change['new']
     
    imgsized= cv2.resize(image,(300,300))
    # compute all detected objects
    detections = model(imgsized)
    
    detected_humans = [(d) for d in detections[0] if d['label'] == int(label_widget.value)]
    min_human_depth  = 9999999
    closest_human = None
    
    for human in detected_humans:
        bbox = human['bbox']
#         RESCALE BOX
        left = bbox[0]
        top = bbox[1]
        right = bbox[2]
        bottom = bbox[3]
        scaled_left = int(width * left)
        scaled_top = int(height * top)
        scaled_right = int(width * right)
        scaled_bottom = int(height * bottom)
        scaled_bbox=  [(scaled_left, scaled_top), (scaled_right, scaled_bottom)]
        scaled_height = scaled_bottom - scaled_top
        scaled_width = scaled_right - scaled_left
        human_depth_image = camera.depth_image[scaled_top:scaled_bottom, scaled_left:scaled_right]
        human_depth = np.average((human_depth_image))
        if human_depth < min_human_depth:
            min_human_depth = human_depth
            closest_human = human
#     IMAGE = image
    
    if (len(detected_humans) == 0):
        robot.left(FOLLOW_TURN_SPEED/2)
    else:
        human = closest_human
        bbox = human['bbox']
#         RESCALE BOX
        left = bbox[0]
        top = bbox[1]
        right = bbox[2]
        bottom = bbox[3]
        scaled_left = int(width * left)
        scaled_top = int(height * top)
        scaled_right = int(width * right)
        scaled_bottom = int(height * bottom)
        scaled_bbox=  [(scaled_left, scaled_top), (scaled_right, scaled_bottom)]
        scaled_height = scaled_bottom - scaled_top
        scaled_width = scaled_right - scaled_left
#         print(scaled_left,scaled_top,scaled_right,scaled_bottom)
#         DRAW BOX ON DEPTH IMAGE
        cv2.rectangle(IMAGE, *scaled_bbox , (255, 0, 0), 2)
        
#         CHECKING IF DISTANCE FROM LEFT IS GREATER THAN THE RIGHT
# TODO: STOP REDUNANT CALLS TO LEFT/FORWARD/RIGHT
        side_diff = left - 1 + right
#         print(side_diff)
        print(min_human_depth)
        human_in_range = min_human_depth < DEPTH_THRESHOLD
        
        if (side_diff > SIDE_DIFF_TURN_THRESHOLD):
            if human_in_range:
                robot.right(FOLLOW_TURN_SPEED)
            else:
                robot.forward_right(FOLLOW_SPEED)
            print("TURNING RIGHT")
        elif (side_diff < -SIDE_DIFF_TURN_THRESHOLD):
            if human_in_range:
                robot.left(FOLLOW_TURN_SPEED)
            else:
                robot.forward_left(FOLLOW_SPEED)
            print("TURNING LEFT")
        else:
#             image_widget.value = bgr8_to_jpeg(camera.depth_image)
#             human_in_range =  depth_image[scaled_top+int(scaled_height/2)][scaled_left+int(scaled_width/2)][0] < DEPTH_THRESHOLD:
#             STRIDE = 1000000
#             human_in_range = False
#             for i in range(int(scaled_height/STRIDE)):
#                 for j in range(int(scaled_width/STRIDE)):
# # # #                 for x in range(scaled_width/STRIDE):
# #                     print("BOX_COORDS", i,j)
#                     y = scaled_top + i * STRIDE
#                     x = scaled_left + j * STRIDE
#                     d = depth_image[y][x]
# #                     print(d)
#                     if d > 0 and d < DEPTH_THRESHOLD:
#                         human_in_range= True
#                         break
#                 if human_in_range:
#                     break

            if min_human_depth < DEPTH_THRESHOLD_MIN:
                robot.backward(FOLLOW_SPEED)
                print("REVERSING")
            elif min_human_depth > DEPTH_THRESHOLD_MAX:
                robot.forward(FOLLOW_SPEED)
                print("FOLLOWING")
            else:
                robot.stop()
                print("STOPPING AT SAFE DISTANCE")
                
        
    image_widget.value = bgr8_to_jpeg(IMAGE)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
processing({'new': camera.color_value})
camera.observe(processing, names='color_value')

STOPPING AT SAFE DISTANCE
1696.1615896072012
REVERSING
STOPPING AT SAFE DISTANCE
1697.5240380540724
REVERSING
STOPPING AT SAFE DISTANCE
1673.5085111086548
REVERSING
STOPPING AT SAFE DISTANCE


1663.9044184821746
REVERSING
1648.3296670356024
REVERSING
1668.1926834130782
REVERSING
1661.6003939636753
REVERSING
1677.0402231223552
REVERSING
1755.0394029850747
REVERSING
1659.352110694184
REVERSING
1683.5058263318147
REVERSING
1615.4709592405904
REVERSING
1654.861923916187
REVERSING
1764.2304298087922
REVERSING
1788.4511315417255
REVERSING
1737.8982594048287
REVERSING
1820.8938248910674
STOPPING AT SAFE DISTANCE
1857.5033842070338
STOPPING AT SAFE DISTANCE
1888.2527189940313
STOPPING AT SAFE DISTANCE
1884.6371376811594
STOPPING AT SAFE DISTANCE
1955.5348697252903
STOPPING AT SAFE DISTANCE
1921.6324148346207
STOPPING AT SAFE DISTANCE
1944.6340427603159
STOPPING AT SAFE DISTANCE
1935.5912018738106
STOPPING AT SAFE DISTANCE
1927.5883585858585
STOPPING AT SAFE DISTANCE
1930.5865573207807
STOPPING AT SAFE DISTANCE
1933.5708037310908
STOPPING AT SAFE DISTANCE
1922.9365796357295
STOPPING AT SAFE DISTANCE
1911.3195813259642
STOPPING AT SAFE DISTANCE
1928.9596274383457
STOPPING AT SAFE DIST

The following code can be used to stop the capturing of the image and the moving of the robot

In [5]:
camera.unobserve_all()
time.sleep(1.0)
robot.stop()